In [3]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program, load_all_programs
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# CHANGE ME!
TASK_ID = 'youngest_over_35'
AUTHOR = 'will'

In [5]:
task = Task(
    id=TASK_ID,
    description="Find the name of the youngest person over 35",
    plan=[
        Plan(id="filter", description="Filter >35"),
        Plan(id="min", description="Min by age"),
        Plan(id="name", description="Get the name"),
    ],
    sample_input={
        "people": [
            {"age": 35, "name": "John"},
            {"age": 36, "name": "Mary"},
            {"age": 37, "name": "Jane"},
        ]
    },
    sample_output="Mary",
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [6]:
python_imperative_fused = replace(prototype,
    language='python-imperative',
    implementation='fused',
    source='''def youngest_over_35(people):
  youngest = None
  for person in people:
    if person['age'] > 35 and (youngest is None or person['age'] < youngest['age']):
      youngest = person
  return youngest['name'] if youngest is not None else None''')
python_imperative_fused.execute(task)
python_imperative_fused.save()

In [7]:
python_imperative_separate = replace(prototype,
    language='python-imperative',
    implementation='separate',
    source='''def youngest_over_35(people):
  over_35 = []
  for person in people:
    if person['age'] > 35:
      over_35.append(person)
    
  youngest = None
  for person in over_35:
    if youngest is None or person['age'] < youngest['age']:
      youngest = person
  return youngest['name'] if youngest is not None else None''')
python_imperative_separate.execute(task)
python_imperative_separate.save()

In [8]:
python_functional = replace(prototype,
    language='python-functional',
    source='''def youngest_over_35(people):
  over_35 = [p for p in people if p['age'] > 35]
  youngest = min(over_35, default=None, key=lambda p: p['age'])
  return youngest['name'] if youngest is not None else None''')
python_functional.execute(task)
python_functional.save()

In [9]:
python_pandas = replace(prototype,
    language='python-pandas',
    source='''def youngest_over_35(people):
  over_35 = people[people.age > 35]
  youngest = over_35.loc[over_35.age.idxmin()]
  return youngest['name']''')
python_pandas.execute(task)
python_pandas.save()

In [10]:
sql = replace(prototype,
    language='sql',
    source='''SELECT name
FROM people
WHERE age = (
  SELECT MIN(age)
  FROM people
  WHERE age > 35)
LIMIT 1''')
sql.execute(task)
sql.save()

In [11]:
datalog = replace(prototype,
    language='datalog',
    source='''
youngest_over_35(name) :- people(age, name), age = min age : { people(age, _), age > 35 }.
    ''')
datalog.execute(task)
datalog.save()